In [1]:
# 1 · install required libraries
# !pip -q install --upgrade plotly anywidget pandas polars numpy


print("✅ Libraries ready & custom-widget manager enabled")

✅ Libraries ready & custom-widget manager enabled


In [1]:
# 2 · ensure project code is importable and reload edited modules
import sys, pathlib, importlib, pprint

root = pathlib.Path(".").resolve()
if str(root) not in sys.path:
    sys.path.append(str(root))

import helpers.data_loader as dl
import widget.nmf_widget  as nmfw
importlib.reload(dl)
importlib.reload(nmfw)

print("✅ helpers & widget packages reloaded from", root)

✅ helpers & widget packages reloaded from /net/users/voy/nmfwidget


In [2]:
# 3 · quick data sanity-check (optional)
H, ids, cancers = dl.get_prepared_data("config.json")
print("Matrix shape:", H.shape)
print("Sample IDs  :", len(ids))
print("Cancer types:", len(cancers))

Matrix shape: (410, 16)
Sample IDs  : 410
Cancer types: 410


In [3]:
# 4 · build & display the widget
from widget import create_nmf_widget

w = create_nmf_widget("config.json")
display(w)          # ‹— anywidget wrapper

# fallback that always works (optional):
# w.figure.show()

NMFHeatmapWidget: Initializing...
NMFHeatmapWidget: Creating figure from config: config.json
Loading component colors from: nmf_component_color_map.json
Loaded component colors: {'Comp_0': '#332288', 'Comp_1': '#88CCEE', 'Comp_2': '#117733', 'Comp_3': '#DDCC77', 'Comp_4': '#CC6677', 'Comp_5': '#882255', 'Comp_6': '#AA4499', 'Comp_7': '#44AA99', 'Comp_8': '#999933', 'Comp_9': '#DDDDDD', 'Comp_10': '#E69F00', 'Comp_11': '#56B4E9', 'Comp_12': '#009E73', 'Comp_13': '#F0E442', 'Comp_14': '#0072B2', 'Comp_15': '#D55E00'}
Component color mapping: {'Component 1': {'color': '#332288', 'matched_key': 'Comp_0'}, 'Component 2': {'color': '#88CCEE', 'matched_key': 'Comp_1'}, 'Component 7': {'color': '#AA4499', 'matched_key': 'Comp_6'}, 'Component 11': {'color': '#E69F00', 'matched_key': 'Comp_10'}, 'Component 3': {'color': '#117733', 'matched_key': 'Comp_2'}, 'Component 5': {'color': '#CC6677', 'matched_key': 'Comp_4'}, 'Component 8': {'color': '#44AA99', 'matched_key': 'Comp_7'}, 'Component 9': {'

NMFHeatmapWidget()

In [5]:
# w.figure.show()

FigureWidget({
    'data': [{'colorbar': {'title': {'text': 'Activity'}, 'x': 1.02},
              'colorscale': [[0.0, '#30123b'], [0.07142857142857142, '#4145ab'],
                             [0.14285714285714285, '#4675ed'],
                             [0.21428571428571427, '#39a2fc'], [0.2857142857142857,
                             '#1bcfd4'], [0.35714285714285715, '#24eca6'],
                             [0.42857142857142855, '#61fc6c'], [0.5, '#a4fc3b'],
                             [0.5714285714285714, '#d1e834'], [0.6428571428571429,
                             '#f3c63a'], [0.7142857142857143, '#fe9b2d'],
                             [0.7857142857142857, '#f36315'], [0.8571428571428571,
                             '#d93806'], [0.9285714285714286, '#b11901'], [1.0,
                             '#7a0402']],
              'hovertemplate': 'Sample: %{x}<br>Component: %{y}<br>Activity: %{z}<extra></extra>',
              'showscale': False,
              'type': 'heatmap',
   

In [6]:
import plotly.io as pio
pio.renderers.default = "notebook"
w.figure.show()

FigureWidget({
    'data': [{'colorbar': {'title': {'text': 'Activity'}, 'x': 1.02},
              'colorscale': [[0.0, '#30123b'], [0.07142857142857142, '#4145ab'],
                             [0.14285714285714285, '#4675ed'],
                             [0.21428571428571427, '#39a2fc'], [0.2857142857142857,
                             '#1bcfd4'], [0.35714285714285715, '#24eca6'],
                             [0.42857142857142855, '#61fc6c'], [0.5, '#a4fc3b'],
                             [0.5714285714285714, '#d1e834'], [0.6428571428571429,
                             '#f3c63a'], [0.7142857142857143, '#fe9b2d'],
                             [0.7857142857142857, '#f36315'], [0.8571428571428571,
                             '#d93806'], [0.9285714285714286, '#b11901'], [1.0,
                             '#7a0402']],
              'hovertemplate': 'Sample: %{x}<br>Component: %{y}<br>Activity: %{z}<extra></extra>',
              'showscale': False,
              'type': 'heatmap',
   

In [7]:
# Check available renderers
import plotly.io as pio
print(pio.renderers)

# Try forcing the notebook renderer
pio.renderers.default = "notebook_connected"
w.figure.show()

# Check if widget frontend is registered
from ipywidgets import Widget
print(Widget.widget_types)

Renderers configuration
-----------------------
    Default renderer: 'notebook'
    Available renderers:
        ['plotly_mimetype', 'jupyterlab', 'nteract', 'vscode',
         'notebook', 'notebook_connected', 'kaggle', 'azure', 'colab',
         'cocalc', 'databricks', 'json', 'png', 'jpeg', 'jpg', 'svg',
         'pdf', 'browser', 'firefox', 'chrome', 'chromium', 'iframe',
         'iframe_connected', 'sphinx_gallery', 'sphinx_gallery_png']



In [8]:
# Save as HTML and view in browser
w.figure.write_html("nmf_heatmap.html")

# Or use static image export
# w.figure.write_image("nmf_heatmap.png", width=1200, height=800)

In [9]:
# Export interactive HTML with sorting controls
from widget.nmf_widget import create_html_with_sorting_controls
html_file = create_html_with_sorting_controls(w, "interactive_nmf.html")
print(f"Interactive HTML with sorting saved to {html_file}")

Interactive HTML with sorting saved to interactive_nmf.html
